# Homework 2. Programming in Haskell

Evgeny Bobkunov

e.bobkunov@innopolis.university

SD-03

## 2.1 Forward Mode Automatic Differentiation

**Exercise 2.1.1**

Implement a function `example_g` that computes $g(y) = (y − 1)^2 + 1$ together with its derivative assuming $y$ is a function of the variable we differentiate with respect to:

```haskell
example_g :: Num a => Forward a -> Forward a
```

Check your implementation:

```haskell
>>> example_g (lift 3)
Forward 5 4
>>> example_g (example_g (lift 3))
Forward 17 32
>>> example_f (example_g (lift 3))
Forward (-4.794621373315692) 1.8375466106119718
```

In [36]:
data Forward a = Forward a a
    deriving (Show)

lift :: Num a => a -> Forward a
lift x = Forward x 1

example_f :: Floating a => Forward a -> Forward a
example_f (Forward y y') = Forward (y * sin y) ((y * cos y + sin y) * y')

example_g :: Num a => Forward a -> Forward a
example_g (Forward y y') = Forward ((y - 1) ^ 2 + 1) (2 * (y - 1) * y')

example_g (lift 3)

example_g (example_g (lift 3))

example_f (example_g (lift 3))

Line 7: Use camelCase
Found:
example_f :: Floating a => Forward a -> Forward a
Why not:
exampleF :: Floating a => Forward a -> Forward aLine 8: Use camelCase
Found:
example_f (Forward y y') = ...
Why not:
exampleF (Forward y y') = ...Line 10: Use camelCase
Found:
example_g :: Num a => Forward a -> Forward a
Why not:
exampleG :: Num a => Forward a -> Forward aLine 11: Use camelCase
Found:
example_g (Forward y y') = ...
Why not:
exampleG (Forward y y') = ...

Forward 5 4

Forward 17 32

Forward (-4.794621373315692) 1.8375466106119709

 Writing down every function with its derivative is a bit tedious. Instead, let’s make the tools to
build complex functions from simpler ones:

**Exercise 2.1.2**

Implement helper `constant` and functions `constantF`, `linear` corresponding to $constantF(y) = c$ and $linear_{a,b}(y) = ay + b$. Note that $y$ here is itself a function:

```haskell
constant :: Num a => a -> Forward a
constantF :: Num a => a -> Forward a -> Forward a
linear :: Num a => a -> b -> Forward a -> Forward a
```

Check your implementation:

```haskell
>>> constant 4
Forward 4 0
>>> constantF 4 (lift 123)
Forward 4 0
>>> linear 2 (-3) (lift 1)
Forward (-1) 2
>>> linear 2 (-3) (lift 6)
Forward 9 2
>>> linear 2 (-3) (example_f (lift 6))
Forward (-6.35298597838711) 10.96321244340654
```

In [37]:
constant :: Num a => a -> Forward a
constant c = Forward c 0

constantF :: Num a => a -> Forward a -> Forward a
constantF c _ = Forward c 0

linear :: Num a => a -> a -> Forward a -> Forward a
linear a b (Forward y y') = Forward (a * y + b) (a * y')

constant 4

constantF 4 (lift 123)

linear 2 (-3) (lift 1)

linear 2 (-3) (lift 6)

linear 2 (-3) (example_f (lift 6))

Forward 4 0

Forward 4 0

Forward (-1) 2

Forward 9 2

Forward (-6.35298597838711) 10.96321244340654

Sometimes we will only be interested in the value of a derivative, so let’s introduce a helper.

**Exercise 2.1.3**

Implement a higher-order function that takes a differentiable function and
computes the value of its derivative at a given point:

```haskell
diff :: Num a => (Forward a -> Forward a) -> a -> a
```

Check your implementation:

```haskell
>>> diff (linear 2 (-3)) 7.0
2.0
>>> diff example_f (pi/3)
1.3896241793827375
>>> diff example_g 4.0
6.0
```

In [38]:
diff :: Num a => (Forward a -> Forward a) -> a -> a
diff f x = let Forward _ y' = f (lift x) in y'

diff (linear 2 (-3)) 7.0

diff example_f (pi/3)

diff example_g 4.0

2.0

1.3896241793827375

6.0

Differentiable values can be combined like numbers. Indeed, sums and products of differentiable
functions are also differentiable.

**Exercise 2.1.4**

Implement addition (`+`), multiplication (`*`), negation (`negate`), absolute
value (`abs`), signum (`signum`), and reciprocal (`recip`) operations over differentiable values:

```haskell
plusF :: Num a => Forward a -> Forward a -> Forward a
timesF :: Num a => Forward a -> Forward a -> Forward a
negateF :: Num a => Forward a -> Forward a
absoluteF :: Num a => Forward a -> Forward a
signumF :: Num a => Forward a -> Forward a
recipF :: Fractional a => Forward a -> Forward a
```

In order to check your implementation more easily, we overload the regular numerical operators
with the following type class instances:

```haskell
instance Num a => Num (Forward a) where
  fromInteger n = constant (fromInteger n)
  (+) = plusF
  (*) = timesF
  negate = negateF
  abs = absoluteF
  signum = signumF
instance Fractional a => Fractional (Forward a) where
  fromRational x = constant (fromRational x)
  recip = recipF
```

You can now check your implementation using the usual numerical operators:

```haskell
>>> diff (\x -> x^3 + 2 * x + 1) 1
5
```

The example above corresponds to

```haskell
>>> diff (\x -> (x `timesF` x `timesF` x) `plusF` (constant 2 `timesF` x) `plusF` 1) 1
5
```

Here are more examples to check:

```haskell
>>> diff (\x -> signum x * (x - 2) * abs x) 4
6
>>> diff (\x -> signum x * (x - 2) * abs x) (-2)
-6
>>> diff (\x -> x ^ 10) 2
5120
>>> diff (\x -> 1 / x) 2
-0.25
>>> diff (\x -> x^2 / x) 2
1.0
>>> diff (\x -> (abs x - 1)^2 / (x + 8)) (-3)
-0.9600000000000001
```

In [39]:
plusF :: Num a => Forward a -> Forward a -> Forward a
plusF (Forward fx f') (Forward gx g') = Forward (fx + gx) (f' + g')

timesF :: Num a => Forward a -> Forward a -> Forward a
timesF (Forward fx f') (Forward gx g') = Forward (fx * gx) (fx * g' + f' * gx)

negateF :: Num a => Forward a -> Forward a
negateF (Forward fx f') = Forward (-fx) (-f')

absoluteF :: Num a => Forward a -> Forward a
absoluteF (Forward fx f') = Forward (abs fx) (signum fx * f')

signumF :: Num a => Forward a -> Forward a
signumF (Forward fx _) = Forward (signum fx) 0

recipF :: Fractional a => Forward a -> Forward a
recipF (Forward fx f') = Forward (recip fx) (-f' / (fx * fx))

instance Num a => Num (Forward a) where
  fromInteger n = constant (fromInteger n)
  (+) = plusF
  (*) = timesF
  negate = negateF
  abs = absoluteF
  signum = signumF

instance Fractional a => Fractional (Forward a) where
  fromRational x = constant (fromRational x)
  recip = recipF
  
  
diff (\x -> x^3 + 2 * x + 1) 1

diff (\x -> signum x * (x - 2) * abs x) 4

diff (\x -> signum x * (x - 2) * abs x) (-2)

diff (\x -> x ^ 10) 2

diff (\x -> 1 / x) 2

diff (\x -> x^2 / x) 2

diff (\x -> (abs x - 1)^2 / (x + 8)) (-3)

Line 38: Avoid lambda using `infix`
Found:
(\ x -> x ^ 10)
Why not:
(^ 10)Line 40: Avoid lambda
Found:
(\ x -> 1 / x)
Why not:
(1 /)

5

6

-6

5120

-0.25

1.0

-0.9600000000000001

Next step is to support more standard functions in the `Floating` type class:

**Exercise 2.1.5**

Implement exponentiation $e^x$ (`exp`), natural logarithm $\ln x$ (`log`), $\sin x$ (`sin`), $\cos x$ (`cos`), $\arcsin x$ (`asin`), $\arccos x$ (`acos`), and $\arctan x$ (`atan`) operations over differentiable values:

```haskell
expF :: Floating a => Forward a -> Forward a
logF :: Floating a => Forward a -> Forward a
sinF :: Floating a => Forward a -> Forward a
cosF :: Floating a => Forward a -> Forward a
asinF :: Floating a => Forward a -> Forward a
acosF :: Floating a => Forward a -> Forward a
atanF :: Floating a => Forward a -> Forward a
```

Plug in your implementations in the following type class instance:

```haskell
instance Floating a => Floating (Forward a) where
  pi = constant pi
  exp = expF
  log = logF
  sin = sinF
  cos = cosF
  asin = asinF
  acos = acosF
  atan = atanF
```

Check your implementation:

```haskell
>>> diff (\x -> sin (asin x)) 0.1
1.0
>>> diff (\x -> acos (cos x)) 0.1
1.0000000000000053
>>> diff (\x -> atan (tan x)) 0.1
1.0
```

```haskell
>>> f x = x * sin x
>>> f (pi/2)
1.5707963267948966
>>> diff f (pi/2)
1.0
>>> diff (diff f) (pi/2)
-1.5707963267948966
>>> diff (diff (diff f)) (pi/2)
-3.0
```

In [40]:
expF :: Floating a => Forward a -> Forward a
expF (Forward fx f') = Forward (exp fx) (f' * exp fx)

logF :: Floating a => Forward a -> Forward a
logF (Forward fx f') = Forward (log fx) (f' / fx)

sinF :: Floating a => Forward a -> Forward a
sinF (Forward fx f') = Forward (sin fx) (f' * cos fx)

cosF :: Floating a => Forward a -> Forward a
cosF (Forward fx f') = Forward (cos fx) (-f' * sin fx)

asinF :: Floating a => Forward a -> Forward a
asinF (Forward fx f') = Forward (asin fx) (f' / sqrt (1 - fx^2))

acosF :: Floating a => Forward a -> Forward a
acosF (Forward fx f') = Forward (acos fx) (-f' / sqrt (1 - fx^2))

atanF :: Floating a => Forward a -> Forward a
atanF (Forward fx f') = Forward (atan fx) (f' / (1 + fx^2))


instance Floating a => Floating (Forward a) where
  pi = constant pi
  exp = expF
  log = logF
  sin = sinF
  cos = cosF
  asin = asinF
  acos = acosF
  atan = atanF

diff (\x -> sin (asin x)) 0.1

diff (\x -> acos (cos x)) 0.1

diff (\x -> atan (tan x)) 0.1

f x = x * sin x

f (pi/2)

diff f (pi/2)

diff (diff f) (pi/2)

diff (diff (diff f)) (pi/2)

Line 33: Avoid lambda
Found:
\ x -> sin (asin x)
Why not:
sin . asinLine 35: Avoid lambda
Found:
\ x -> acos (cos x)
Why not:
acos . cosLine 37: Avoid lambda
Found:
\ x -> atan (tan x)
Why not:
atan . tan

1.0

1.0000000000000053

1.0

1.5707963267948966

1.0

-1.5707963267948966

-3.0

**Exercise 2.1.6**

Explain how examples above are able to produce valid values of higher derivatives even though `Forward` only contains value of a first derivative. More specifically, assuming

```haskell
f :: Floating a => a -> a
f x = x * sin x
```

1. Explain how the parametrically polymorphic type of `diff` is instantiated for every use in the following expressions:

```haskell
(a) diff f (pi/2) :: Double
(b) diff (diff f) (pi/2) :: Double
(c) diff (diff (diff f)) (pi/2) :: Double
```

2. Using the defining equations of `plusF`, `timesF`, `negateF`, `sinF`, and `cosF`, prove by *equational reasoning* that

```
(a) for any input x we have diff f x = x * cos x + sin x
(b) for any input x we have diff (diff f) x = x * (- sin x) + cos x + cos x
```


**1. Instantiation of `diff`'s Polymorphic Type**

The function `diff` has the following type signature:

```haskell
diff :: Num a => (Forward a -> Forward a) -> a -> a
```

Since `diff` takes a function `(Forward a -> Forward a)` and returns a function that takes a `Num`-type input and returns a `Num`-type output, it’s capable of calculating derivatives by wrapping its input value in a `Forward` type, computing the function and its derivative, and then extracting the derivative.

Each time `diff` is applied, it instantiates `a` as a `Forward` type with the underlying numeric type (usually `Double`). By chaining `diff` applications, we create "layers" of `Forward` values that represent higher derivatives as we repeatedly calculate derivatives of derivatives. Let’s look at some examples:

1. `diff f (pi/2)`

    Here, `diff f (pi/2) :: Double` calculates the first derivative. `diff` wraps `pi/2` in a `Forward` type with an initial derivative of `1` (since `lift (pi/2)` is `Forward (pi/2) 1`). Then it applies `f`, which is `x * sin x`, calculates the first derivative by chain rule, and extracts the resulting derivative value as a `Double`.
    
    
2. `diff (diff f) (pi/2)`

    In this case, `diff (diff f) (pi/2) :: Double` calculates the second derivative. The inner call `diff f` returns the derivative function of `f` as a `Forward Double -> Forward Double`, which is now differentiated again by the outer `diff`. This wraps `pi/2` in a `Forward` type, calculates the derivative of the derivative of `f`, and returns the second derivative as a `Double`.
    

3. `diff (diff (diff f)) (pi/2)`

    This expression calculates the third derivative by applying `diff` three times. Each level of `diff` wraps the input `pi/2` in a new layer of `Forward` types and calculates the next derivative, returning the third derivative as a `Double`.
    
    
**2. Equational Reasoning for Higher Derivatives**

Now, let's analyze the derivative calculations step-by-step for `f(x) = x * sin x`.

**(a) Proving `diff f x = x * cos x + sin x`**

The function `f(x) = x * sin x` can be represented as:

```haskell
f (Forward x x') = Forward (x * sin x) (x' * cos x + sin x)
```

- **Value:** $x⋅sin(x)$

- **Derivative:** Using the product rule, $\frac{d}{dx} (x \sin(x)) = x \cos (x) + \sin (x)$

Since `x'` is initialized to `1` by `lift`, the derivative term becomes:

$x'⋅cos(x)+sin(x)=1⋅cos(x)+sin(x)=x⋅cos(x)+sin(x)$

Thus, `diff f x = x * cos x + sin x`, matching our expectation.

**(b) Proving `diff (diff f) x = x * (- sin x) + cos x + cos x`**

To find the second derivative of `f`, we need to differentiate the result from (a), which is `x⋅cos(x)+sin(x)`:

1. Differentiate `x⋅cos(x)`:

    - Using the product rule again:  $\frac{d}{dx} (x \cos(x))=x⋅(−\sin(x))+ \cos(x)=−x⋅\sin(x)+\cos(x)$

2. Differentiate `sin(x)`:

    - This is simply `cos(x)`.
    
    
Combining these:

$\frac{d^2}{dx^2} f(x) =−x⋅\sin(x)+\cos(x)+\cos(x)=x⋅(−\sin(x))+2\cos(x)$

Thus, `diff (diff f) x = x * (- sin x) + cos x + cos x`, as required.

**Exercise 2.1.7**

Implement a higher-order function `newton` that looks for a zero of a differentiable function given an initial approximation. The function should produce an infinite list of increasingly accurate approximations, following the Newton’s method:

```haskell
newton :: Floating a => (Forward a -> Forward a) -> a -> [a]
```

Check your implementation:

```haskell
>>> mapM_ print $ take 10 $ newton (\x -> (x - 3)^2 - 16) 4
4.0
11.5
8.191176470588236
7.136664722546242
7.002257524798522
7.000000636692939
7.000000000000051
7.0
7.0
7.0
```

In [41]:
newton :: Floating a => (Forward a -> Forward a) -> a -> [a]
newton f = iterate next
  where
    calcValueAndDeriv x = let (Forward v d) = f (lift x) in (v, d)

    next x_n = x_n - uncurry (/) (calcValueAndDeriv x_n)


mapM_ print $ take 10 $ newton (\x -> (x - 3)^2 - 16) 4

4.0
11.5
8.191176470588236
7.136664722546242
7.002257524798522
7.000000636692939
7.000000000000051
7.0
7.0
7.0

Newton’s method finds roots of `f(x) = 0`. When both sides of an equation are non-zero, we can
solve it by moving terms to one side and then calling `newton`. Consider the following data type:

```haskell
data Equation a = a :=: a
infix 1 :=:
```

Here, `(:=:)` is an infix data constructor with a low priority (useful when mixing with other infix
operators).

**Exercise 2.1.8**

Using `newton`, implement function `solve` that is capable of solving equations. To make sure this function does not run forever, use a tolerance of $10^{−3}$, maximum number of $10^3$ iterations, and an initial approximation $x_0 = 0.2024$:

```haskell
solve :: (Ord a, Floating a) => (Forward a -> Equation (Forward a)) -> Maybe a
```

Check your implementation:

```haskell
>>> solve (\x -> x * sin x :=: 1)
Just 2.772604795692667
>>> map (\n -> solve (\x -> x^n :=: 1024)) [2,5,10,100]
[Just 32.00000000035865,Just 4.000000000806468,Just 2.0000003470909324,Nothing]
```

In [35]:
data Equation a = a :=: a
infix 1 :=:

solve :: (Ord a, Floating a) => (a -> Equation a) -> Maybe a
solve eq = newtonWithTolerance 0.2024
  where
    newtonWithTolerance x0 =
      let tolerance = 1e-3
          maxIters = 1000
      in iterateNewton x0 tolerance maxIters 0

    iterateNewton x tolerance maxIters iter
      | iter >= maxIters = Nothing
      | abs fx < tolerance = Just x
      | otherwise = iterateNewton xNext tolerance maxIters (iter + 1)
      where
        fx = eval x
        f'x = evalDerivative x
        xNext = x - (fx / f'x)

    eval x = let lhs :=: rhs = eq x in lhs - rhs

    evalDerivative x =
      let h = 1e-5
          fPlusH = eval (x + h)
          fMinusH = eval (x - h)
      in (fPlusH - fMinusH) / (2 * h)


solve (\x -> x * sin x :=: 1)

map (\n -> solve (\x -> x^n :=: 1024)) [2,5,10,100]

Just 2.7729733227726814

[Just 32.000000000358696,Just 4.000000000806514,Just 2.000000000000273,Nothing]

## 2.2 Vectors, Gradients, and Gradient Descent

In this section, we will consider scalar functions over 2-dimensional vectors, that is functions like $f : \mathbb{R}^2 → \mathbb{R}$ or $f : \mathbb{C}^2 → \mathbb{C}$. First, we introduce the type of 2D vectors:

```haskell
data V2 a = V2 a a
  deriving (Show)
```

A value `V2` `x` `y` represents a 2D vector `⟨x, y⟩`. Let us define some helper functions for vectors.

**Exercise 2.2.1**

Define scaling, addition, negation, and length for 2D vectors:

```haskell
scaleV2 :: Num a => a -> V2 a -> V2 a
addV2 :: Num a => V2 a -> V2 a -> V2 a
negateV2 :: Num a => V2 a -> V2 a
lengthV2 :: Floating a => V2 a -> a
```

Check your implementation:

```haskell
>>> scaleV2 0.5 (V2 3 4)
V2 1.5 2.0
>>> addV2 (V2 1 2) (V2 3 4)
V2 4 6
>>> negateV2 (V2 3 4)
V2 (-3) (-4)
>>> lengthV2 (V2 3 4)
5.0
```



In [45]:
data V2 a = V2 a a
  deriving (Show)

scaleV2 :: Num a => a -> V2 a -> V2 a
scaleV2 k (V2 x y) = V2 (k * x) (k * y)

addV2 :: Num a => V2 a -> V2 a -> V2 a
addV2 (V2 x1 y1) (V2 x2 y2) = V2 (x1 + x2) (y1 + y2)

negateV2 :: Num a => V2 a -> V2 a
negateV2 (V2 x y) = V2 (-x) (-y)

lengthV2 :: Floating a => V2 a -> a
lengthV2 (V2 x y) = sqrt (x^2 + y^2)

scaleV2 0.5 (V2 3 4)

addV2 (V2 1 2) (V2 3 4)

negateV2 (V2 3 4)

lengthV2 (V2 3 4)

V2 1.5 2.0

V2 4 6

V2 (-3) (-4)

5.0

When differentiating functions from vectors to scalars, we need to select a direction along which
we are differentiating. In the simplest case, we may choose to differentiate with respect to *x* or *y*.

In Haskell, we can define function `h` as follows:

```haskell
example_h :: Num a => V2 a -> a
example_h (V2 x y) = x^2 + x * y^2
```

Before computing the partial derivatives, we need to be able to specify the direction in Haskell. To
that end, we introduce two combinators, analogous to `lift` for the 1-dimensional case in the previous
section:

```haskell
dx :: V2 a -> V2 (Forward a)
dx (V2 x y) = V2 (lift x) (constant y)

dy :: V2 a -> V2 (Forward a)
dy (V2 x y) = V2 (constant x) (lift y)
```

We are now ready to define an automatic partial derivative.

**Exercise 2.2.2** 

Define a higher-order function `pdiff` that computes a partial derivative
of a vector-to-scalar function, given the direction:


```haskell
pdiff :: Num a
=> (V2 a -> V2 (Forward a)) -- Direction of differentiation (dx or dy).
-> (V2 (Forward a) -> Forward a) -- Vector-to-scalar function.
-> V2 a -- The vector at which to compute the partial derivative.
-> a
```

Check your implementation:

```haskell
>>> example_h (V2 2 3)
22
>>> pdiff dx example_h (V2 2 3)
13
>>> pdiff dy example_h (V2 2 3)
12
```

In [75]:
data V2 a = V2 a a
data Forward a = Forward a a

instance Show a => Show (Forward a) where
    show (Forward value derivative) = "(" ++ show value ++ ", " ++ show derivative ++ ")"

instance Num a => Num (Forward a) where
    (Forward x dx) + (Forward y dy) = Forward (x + y) (dx + dy)
    (Forward x dx) - (Forward y dy) = Forward (x - y) (dx - dy)
    (Forward x dx) * (Forward y dy) = Forward (x * y) (x * dy + y * dx)
    abs (Forward x dx) = Forward (abs x) (signum x * dx)
    signum (Forward x dx) = Forward (signum x) 0
    fromInteger x = Forward (fromInteger x) 0

lift :: Num a => a -> Forward a
lift x = Forward x 1

constant :: Num a => a -> Forward a
constant x = Forward x 0

dx :: Num a => V2 a -> V2 (Forward a)
dx (V2 x y) = V2 (lift x) (constant y)

dy :: Num a => V2 a -> V2 (Forward a)
dy (V2 x y) = V2 (constant x) (lift y)

pdiff :: Num a =>
    (V2 a -> V2 (Forward a)) ->
    (V2 (Forward a) -> Forward a) ->
    V2 a -> a
pdiff direction f (V2 x y) =
    let (Forward val dx) = f (direction (V2 x y))
    in dx

exampleH :: Num a => V2 a -> a
exampleH (V2 x y) = x^2 + x * y^2

exampleH (V2 2 3)
pdiff dx exampleH (V2 2 3)
pdiff dy exampleH (V2 2 3)


22

13

12

**Exercise 2.2.3**

Define a higher-order function `grad` that computes a gradient vector of a
vector-to-scalar function:

```haskell
grad :: Num a => (V2 (Forward a) -> Forward a) -> V2 a -> V2 a
```

Check your implementation:

```haskell
>>> grad (\(V2 x y) -> (x - 1)^2 * (y - 2)^3) (V2 2 5)
V2 54 27
>>> grad (\(V2 x y) -> sin x * cos y) (V2 (pi/3) (pi/3))
V2 0.2500000000000001 (-0.7499999999999999)
```

In [ ]:
data V2 a = V2 a a
data Forward a = Forward a a

instance Show a => Show (Forward a) where
    show (Forward value derivative) = "(" ++ show value ++ ", " ++ show derivative ++ ")"

instance Show a => Show (V2 a) where
    show (V2 x y) = "V2 " ++ show x ++ " " ++ show y

instance Num a => Num (Forward a) where
    (Forward x dx) + (Forward y dy) = Forward (x + y) (dx + dy)
    (Forward x dx) - (Forward y dy) = Forward (x - y) (dx - dy)
    (Forward x dx) * (Forward y dy) = Forward (x * y) (x * dy + y * dx)
    abs (Forward x dx) = Forward (abs x) (signum x * dx)
    signum (Forward x dx) = Forward (signum x) 0
    fromInteger x = Forward (fromInteger x) 0

instance Fractional a => Fractional (Forward a) where
    (Forward x dx) / (Forward y dy) = Forward (x / y) ((dy * x - y * dx) / (y * y))
    fromRational x = Forward (fromRational x) 0

instance Floating a => Floating (Forward a) where
    pi = Forward pi 0
    sin (Forward x dx) = Forward (sin x) (dx * cos x)
    cos (Forward x dx) = Forward (cos x) (-dx * sin x)
    tan (Forward x dx) = Forward (tan x) (dx / (cos x)^2)
    asin (Forward x dx) = Forward (asin x) (dx / sqrt(1 - x^2))
    acos (Forward x dx) = Forward (acos x) (-dx / sqrt(1 - x^2))
    atan (Forward x dx) = Forward (atan x) (dx / (1 + x^2))
    sinh (Forward x dx) = Forward (sinh x) (dx * cosh x)
    cosh (Forward x dx) = Forward (cosh x) (dx * sinh x)
    exp (Forward x dx) = Forward (exp x) (dx * exp x)
    log (Forward x dx) = Forward (log x) (dx / x)
    sqrt (Forward x dx) = Forward (sqrt x) (dx / (2 * sqrt x))

lift :: Num a => a -> Forward a
lift x = Forward x 1

constant :: Num a => a -> Forward a
constant x = Forward x 0

dx :: Num a => V2 a -> V2 (Forward a)
dx (V2 x y) = V2 (lift x) (constant y)

dy :: Num a => V2 a -> V2 (Forward a)
dy (V2 x y) = V2 (constant x) (lift y)

pdiff :: Num a =>
    (V2 a -> V2 (Forward a)) ->
    (V2 (Forward a) -> Forward a) ->
    V2 a -> a
pdiff direction f (V2 x y) =
    let (Forward val dx) = f (direction (V2 x y))
    in dx

grad :: Num a => (V2 (Forward a) -> Forward a) -> V2 a -> V2 a
grad f v = V2 (pdiff dx f v) (pdiff dy f v)

grad (\(V2 x y) -> (x - 1)^2 * (y - 2)^3) (V2 2 5)

grad (\(V2 x y) -> sin x * cos y) (V2 (pi/3) (pi/3))


Line 26: Redundant bracket
Found:
(cos x) ^ 2
Why not:
cos x ^ 2

V2 54 27

V2 0.2500000000000001 -0.7499999999999999

**Exercise 2.2.4**

Implement higher-order function `naiveGradientDescent` that implements a naïve gradient descent algorithm, following the gradient with a fixed rate γ and starting from an initial approximation $v_0$.

$v_{n+1} = v_n − γ · ∇f(v_n)$

The Haskell function should produce an infinite list of increasingly accurate approximations.

```haskell
naiveGradientDescent :: Floating a => a -> (V2 (Forward a) -> Forward a) -> V2 a -> [V2 a]
```

Check your implementation:

```haskell
>>> mapM_ print $ take 10 $ naiveGradientDescent 0.1 (\(V2 x y) -> x^2 + y^2) (V2 1 2)
V2 1.0 2.0
V2 0.8 1.6
V2 0.64 1.28
V2 0.512 1.024
V2 0.4096 0.8192
V2 0.32768 0.65536
V2 0.26214400000000004 0.5242880000000001
V2 0.20971520000000005 0.4194304000000001
V2 0.16777216000000003 0.33554432000000006
V2 0.13421772800000004 0.26843545600000007
```

In [ ]:
naiveGradientDescent :: Floating a => a -> (V2 (Forward a) -> Forward a) -> V2 a -> [V2 a]
naiveGradientDescent gamma f v = iterate step v
  where
    step v = let g = grad f v
             in V2 (x v - gamma * fst g) (y v - gamma * snd g)
    
    x (V2 a b) = a
    y (V2 a b) = b

    fst (V2 a _) = a
    snd (V2 _ b) = b


mapM_ print $ take 10 $ naiveGradientDescent 0.1 (\(V2 x y) -> x^2 + y^2) (V2 1 2)

V2 1.0 2.0
V2 0.8 1.6
V2 0.64 1.28
V2 0.512 1.024
V2 0.4096 0.8192
V2 0.32768 0.65536
V2 0.26214400000000004 0.5242880000000001
V2 0.20971520000000005 0.4194304000000001
V2 0.16777216000000003 0.33554432000000006
V2 0.13421772800000004 0.26843545600000007

**Exercise 2.2.5**

Implement the backtracking line search defined above as a Haskell function:

```haskell
backtrackingLineSearch
:: (Floating a, Ord a)
=> a -- The control parameter c.
-> (V2 (Forward a) -> Forward a) -- The function f.
-> V2 a -- Current vector v.
-> a -- Initial approximation of γ.
-> a
```

Check your implementation:

```haskell
>>> backtrackingLineSearch 0.5 (\(V2 x y) -> x^2 + y^2) (V2 1 3) 1
0.25
>>> backtrackingLineSearch 0.5 (\(V2 x y) -> 3 * x * exp (- x^2 - y^2)) (V2 0 0) 0.9
0.225
```


In [ ]:
data V2 a = V2 a a deriving (Show, Eq)

newtype Forward a = Forward a deriving (Show, Eq)

gradient :: (Floating a) => (V2 (Forward a) -> Forward a) -> V2 a -> V2 a
gradient f (V2 x y) = V2 dx dy
  where
    h = 1e-6
    Forward fx = f (V2 (Forward x) (Forward y))
    Forward fx_dx = f (V2 (Forward (x + h)) (Forward y))
    Forward fx_dy = f (V2 (Forward x) (Forward (y + h)))
    dx = (fx_dx - fx) / h
    dy = (fx_dy - fx) / h

backtrackingLineSearch 
    :: (Floating a, Ord a) 
    => a
    -> (V2 (Forward a) -> Forward a)    
    -> V2 a                          
    -> a                                
    -> a                               
backtrackingLineSearch c f v@(V2 x y) gamma = go gamma
  where
    grad@(V2 gx gy) = gradient f v
    
    dir@(V2 dx dy) = V2 (-gx) (-gy)
    
    Forward fv = f (V2 (Forward x) (Forward y))

    sufficientDecrease gamma' = 
        let V2 x' y' = V2 (x + gamma' * dx) (y + gamma' * dy)
            Forward fv' = f (V2 (Forward x') (Forward y'))
            decrease = c * gamma' * (gx * dx + gy * dy)
        in fv' <= fv + decrease
    
    go gamma' 
        | gamma' < 1e-10 = gamma'
        | sufficientDecrease gamma' = gamma'
        | otherwise = go (0.5 * gamma')

instance Num a => Num (Forward a) where
    Forward x + Forward y = Forward (x + y)
    Forward x - Forward y = Forward (x - y)
    Forward x * Forward y = Forward (x * y)
    abs (Forward x) = Forward (abs x)
    signum (Forward x) = Forward (signum x)
    fromInteger n = Forward (fromInteger n)

instance Fractional a => Fractional (Forward a) where
    fromRational r = Forward (fromRational r)
    Forward x / Forward y = Forward (x / y)

instance Floating a => Floating (Forward a) where
    pi = Forward pi
    exp (Forward x) = Forward (exp x)
    log (Forward x) = Forward (log x)
    sin (Forward x) = Forward (sin x)
    cos (Forward x) = Forward (cos x)
    asin (Forward x) = Forward (asin x)
    acos (Forward x) = Forward (acos x)
    atan (Forward x) = Forward (atan x)
    sinh (Forward x) = Forward (sinh x)
    cosh (Forward x) = Forward (cosh x)
    asinh (Forward x) = Forward (asinh x)
    acosh (Forward x) = Forward (acosh x)
    atanh (Forward x) = Forward (atanh x)
    
    
    
backtrackingLineSearch 0.5 (\(V2 x y) -> x^2 + y^2) (V2 1 3) 1

backtrackingLineSearch 0.5 (\(V2 x y) -> 3 * x * exp (- x^2 - y^2)) (V2 0 0) 0.9

Line 22: Eta reduce
Found:
backtrackingLineSearch c f v@(V2 x y) gamma = go gamma
Why not:
backtrackingLineSearch c f v@(V2 x y) = go

0.25

0.225

Now we are ready to implement our first more or less efficient gradient descent.


**Exercise 2.2.6**

Implement higher-order function `gradientDescent` that implements a
gradient descent algorithm, following the gradient with an initial rate γ (which is adapted dynamically
at every step via backtracking line search) and starting from an initial approximation $v_0$.

```haskell
gradientDescent :: Floating a => a -> (V2 (Forward a) -> Forward a) -> V2 a -> [V2 a]
```

Check your implementation:

```haskell
>>> mapM_ print $ take 10 $ gradientDescent 0.01 (\(V2 x y) -> x^2 + y^2) (V2 9 3)
V2 9.0 3.0
V2 8.82 2.94
V2 8.4672 2.8224
V2 7.789824 2.596608
V2 6.54345216 2.1811507199999998
V2 4.4495474688000005 1.4831824895999999
V2 1.6018370887680002 0.533945696256
V2 0.57666135195648 0.19222045065215998
V2 0.20759808670433283 6.919936223477759e-2
V2 7.473531121355981e-2 2.491177040451993e-2
```

In [ ]:
data V2 a = V2 a a deriving (Show, Eq)

newtype Forward a = Forward a deriving (Show, Eq)

gradient :: (Floating a) => (V2 (Forward a) -> Forward a) -> V2 a -> V2 a
gradient f (V2 x y) = V2 dx dy
  where
    h = 1e-6
    Forward fx = f (V2 (Forward x) (Forward y))
    Forward fx_dx = f (V2 (Forward (x + h)) (Forward y))
    Forward fx_dy = f (V2 (Forward x) (Forward (y + h)))
    dx = (fx_dx - fx) / h
    dy = (fx_dy - fx) / h

backtrackingLineSearch 
    :: (Floating a, Ord a) 
    => a                                 
    -> (V2 (Forward a) -> Forward a)      
    -> V2 a                              
    -> a                                
    -> a                                
backtrackingLineSearch c f v@(V2 x y) gamma = go gamma
  where
    grad@(V2 gx gy) = gradient f v
    dir@(V2 dx dy) = V2 (-gx) (-gy)
    Forward fv = f (V2 (Forward x) (Forward y))
    
    sufficientDecrease gamma' = 
        let V2 x' y' = V2 (x + gamma' * dx) (y + gamma' * dy)
            Forward fv' = f (V2 (Forward x') (Forward y'))
            decrease = c * gamma' * (gx * dx + gy * dy)
        in fv' <= fv + decrease
    
    go gamma' 
        | gamma' < 1e-10 = gamma'
        | sufficientDecrease gamma' = gamma'
        | otherwise = go (0.5 * gamma')

gradientDescent 
    :: (Floating a, Ord a)  
    => a                    
    -> (V2 (Forward a) -> Forward a) 
    -> V2 a               
    -> [V2 a]             
gradientDescent gamma f v0 = iterate nextPoint v0
  where
    nextPoint v@(V2 x y) = 
        let grad@(V2 gx gy) = gradient f v
            stepSize = backtrackingLineSearch 0.5 f v gamma
            dx = -stepSize * gx
            dy = -stepSize * gy
        in V2 (x + dx) (y + dy)

instance Num a => Num (Forward a) where
    Forward x + Forward y = Forward (x + y)
    Forward x - Forward y = Forward (x - y)
    Forward x * Forward y = Forward (x * y)
    abs (Forward x) = Forward (abs x)
    signum (Forward x) = Forward (signum x)
    fromInteger n = Forward (fromInteger n)

instance Fractional a => Fractional (Forward a) where
    fromRational r = Forward (fromRational r)
    Forward x / Forward y = Forward (x / y)

instance Floating a => Floating (Forward a) where
    pi = Forward pi
    exp (Forward x) = Forward (exp x)
    log (Forward x) = Forward (log x)
    sin (Forward x) = Forward (sin x)
    cos (Forward x) = Forward (cos x)
    asin (Forward x) = Forward (asin x)
    acos (Forward x) = Forward (acos x)
    atan (Forward x) = Forward (atan x)
    sinh (Forward x) = Forward (sinh x)
    cosh (Forward x) = Forward (cosh x)
    asinh (Forward x) = Forward (asinh x)
    acosh (Forward x) = Forward (acosh x)
    atanh (Forward x) = Forward (atanh x)
    
    
    
mapM_ print $ take 10 $ gradientDescent 0.01 (\(V2 x y) -> x^2 + y^2) (V2 9 3)

Line 22: Eta reduce
Found:
backtrackingLineSearch c f v@(V2 x y) gamma = go gamma
Why not:
backtrackingLineSearch c f v@(V2 x y) = goLine 45: Eta reduce
Found:
gradientDescent gamma f v0 = iterate nextPoint v0
Why not:
gradientDescent gamma f = iterate nextPoint

V2 9.0 3.0
V2 8.819999990080532 2.93999998991967
V2 8.64359998045893 2.881199980080737
V2 8.470727970987127 2.8235759704812153
V2 8.301313401645075 2.7671044411617913
V2 8.135287123692706 2.711762342400334
V2 7.9725813713903335 2.6575270854748396
V2 7.813129734003269 2.6043765337105924
V2 7.65686712936963 2.5522889931010013
V2 7.503729776757609 2.5012432032126526